In [2]:
# Import required libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, KFold
from scipy import stats

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set style for plots
plt.style.use('default') 
sns.set_theme()

def load_cbcl_depression():
    """Load depression scores and clean"""
    # Load CBCL data
    cbcl_file = "../data/core/mental-health/mh_p_cbcl.csv"
    cbcl_df = pd.read_csv(cbcl_file, low_memory=False)
    
    # Get 3-year follow-up 
    followup_mask = cbcl_df['eventname'].str.contains('3_year', case=False, na=False)
    followup_df = cbcl_df[followup_mask].copy()
    
    # Get depression score 
    dep_col = [col for col in followup_df.columns if 'depress' in col.lower() and 'dsm5' in col.lower()][0]
    
    # Remove subjects with missing /r invalid (555,999,777) scores
    invalid_scores = [555, 999, 777, np.nan]
    valid_mask = ~followup_df[dep_col].isin(invalid_scores)
    clean_df = followup_df[valid_mask].copy()
    
    # only get necessary columns
    result_df = clean_df[['src_subject_id', dep_col]].copy()
    result_df = result_df.rename(columns={dep_col: 'depression_score'})
    
    print(f"Original subjects: {len(followup_df)}")
    print(f"Subjects after cleaning: {len(result_df)}")
    print(f"\nDepression score statistics:")
    print(result_df['depression_score'].describe())
    
    return result_df

def load_baseline_data(subject_ids):
    """Load baseline data for only  subjects with CBCL dsm5 depress scores. Get all data from all core subfolders, excluding CBCL scores"""
    core_dir = Path("../data/core")
    all_data = []
    
    # Calculate min required subjects (80% of full cohort -- we get rid of files missing over 20%)
    min_subjects = len(subject_ids) * 0.8
    print(f"Minimum required subjects per file: {min_subjects:.0f}")
    
    # Process each subdirectory 
    for subdir in core_dir.iterdir():
        if not subdir.is_dir():
            continue
            
        print(f"\nProcessing {subdir.name}...")
        subdir_data = []
        
        # Skip CBCL directory
        if 'cbcl' in subdir.name.lower():
            print(f"Skipping {subdir.name} (CBCL data excluded)")
            continue
        
        for file in subdir.glob('*.csv'):
            try:
                # don't get CBCL --- too colinear with future CBCL depression 
                if 'cbcl' in file.name.lower():
                    print(f"  - Skipping {file.name} (CBCL data excluded)")
                    continue
                
                # Load data
                df = pd.read_csv(file, low_memory=False)
                
                # Check if file has subject ID column -- fixes issue 
                if 'src_subject_id' not in df.columns:
                    continue
                    
                # Get baseline data
                if 'eventname' in df.columns:
                    baseline_mask = df['eventname'].str.contains('baseline', case=False, na=False)
                    df = df[baseline_mask].copy()
                
                # Filter for our subjects
                df = df[df['src_subject_id'].isin(subject_ids)].copy()
                
                # Only keep files with >80% subjects
                if len(df) >= min_subjects:
                    print(f"  - {file.name}: {len(df)} subjects (keeping)")
                    # Add source information to column names
                    df.columns = [f"{col}_{subdir.name}_{file.stem}" if col not in ['src_subject_id', 'eventname'] else col 
                                for col in df.columns]
                    subdir_data.append(df)
                else:
                    print(f"  - {file.name}: {len(df)} subjects (skipping - too few subjects)")
                
            except Exception as e:
                print(f"  - Error loading {file.name}: {str(e)}")
        
        # Merge data within each subdirectory -- otherwise stalls out 
        if subdir_data:
            print(f"Merging {len(subdir_data)} files from {subdir.name}...")
            subdir_merged = subdir_data[0]
            for i, df in enumerate(subdir_data[1:], 1):
                suffixes = (f'_{i}', f'_{i+1}')
                subdir_merged = pd.merge(subdir_merged, df, on='src_subject_id', how='outer', suffixes=suffixes)
            all_data.append(subdir_merged)
            print(f"Completed {subdir.name} merge. Shape: {subdir_merged.shape}")
    
    # Merge subdirectory dataframes
    if all_data:
        print("\nMerging all subdirectories...")
        final_df = all_data[0]
        for i, df in enumerate(all_data[1:], 1):
            print(f"Merging subdirectory {i+1} of {len(all_data)}...")
            suffixes = (f'_dir{i}', f'_dir{i+1}')
            final_df = pd.merge(final_df, df, on='src_subject_id', how='outer', suffixes=suffixes)
            print(f"Current shape: {final_df.shape}")
        return final_df
    return None

def create_depression_categories(df):
    """Create depression categories using statistical methods to identify high scores"""
    # Calculate statistics
    mean = df['depression_score'].mean()
    std = df['depression_score'].std()
    q1 = df['depression_score'].quantile(0.25)
    q3 = df['depression_score'].quantile(0.75)
    iqr = q3 - q1
    
    # Calculate thresholds using both methods
    z_score_threshold = mean + 2*std  # Scores > 2 SD above mean
    iqr_threshold = q3 + 1.5*iqr      # Scores > Q3 + 1.5*IQR
    
    # Use IQR method (more robust to non-normal distribution)
    high_threshold = iqr_threshold
    
    # Create categories
    df['depression_category'] = pd.cut(
        df['depression_score'],
        bins=[-np.inf, q1, q3, np.inf],
        labels=['low', 'medium', 'high']
    )
    
    # Create binary depression (1 for high, 0 for others)
    df['depression_binary'] = (df['depression_category'] == 'high').astype(int)
    
    # Print distribution information
    print("\nDepression Score Statistics:")
    print(f"Mean: {mean:.2f}")
    print(f"Std: {std:.2f}")
    print(f"Q1 (25th percentile): {q1:.2f}")
    print(f"Q3 (75th percentile): {q3:.2f}")
    print(f"IQR: {iqr:.2f}")
    
    print("\nThresholds:")
    print(f"Z-score method (2 SD above mean): {z_score_threshold:.2f}")
    print(f"IQR method (Q3 + 1.5*IQR): {iqr_threshold:.2f}")
    
    print("\nDepression Categories Distribution:")
    print(df['depression_category'].value_counts(normalize=True))
    
    return df

class DataPreprocessor:
    """
    A class to handle all data preprocessing steps for the ABCD dataset
    """
    def __init__(self, missing_threshold=0.2, 
                 cont_var_threshold=0.01, 
                 cat_var_threshold=0.01,
                 correlation_threshold=0.95,
                 unique_threshold=20,
                 sample_ratio_threshold=0.1,
                 output_dir='../data/processed'):
        """
        Initialize preprocessing parameters
        """
        self.missing_threshold = missing_threshold
        self.cont_var_threshold = cont_var_threshold
        self.cat_var_threshold = cat_var_threshold
        self.correlation_threshold = correlation_threshold
        self.unique_threshold = unique_threshold
        self.sample_ratio_threshold = sample_ratio_threshold
        self.output_dir = output_dir
        self.encoders = {}
        self.scaler = None
        
        # Create output directory if it doesn't exist
        Path(output_dir).mkdir(parents=True, exist_ok=True)
        
    def identify_variable_types(self, df):
        """
        Identify continuous, categorical, and binary variables
        """
        continuous_vars = []
        categorical_vars = []
        binary_vars = []
        
        for col in df.columns:
            if col in ['src_subject_id', 'depression_score']:
                continue
                
            n_unique = df[col].nunique()
            n_samples = len(df)
            unique_ratio = n_unique / n_samples
            is_numeric = pd.api.types.is_numeric_dtype(df[col])
            
            # Binary variables
            if n_unique == 2:
                binary_vars.append(col)
                continue
                
            # Non-numeric variables are categorical
            if not is_numeric:
                categorical_vars.append(col)
                continue
                
            # Numeric variables need further investigation
            if is_numeric:
                is_integer = df[col].dtype in ['int32', 'int64']
                if (is_integer and n_unique <= self.unique_threshold) or \
                   (n_unique <= self.unique_threshold and unique_ratio <= self.sample_ratio_threshold):
                    value_counts = df[col].value_counts(normalize=True)
                    if value_counts.max() < 0.5:  # No single value dominates
                        categorical_vars.append(col)
                    else:
                        continuous_vars.append(col)
                else:
                    continuous_vars.append(col)
        
        return continuous_vars, categorical_vars, binary_vars
    
    def handle_special_values(self, df):
        """
        Convert special values (555, 777, 999) to NaN before missing value analysis
        """
        print("\nHandling special values...")
        special_values = [555, 777, 999]
        special_value_counts = {val: 0 for val in special_values}
        
        # Count special values in each column
        for col in df.columns:
            if col in ['src_subject_id', 'depression_score']:
                continue
            for val in special_values:
                count = (df[col] == val).sum()
                if count > 0:
                    special_value_counts[val] += count
                    print(f"Found {count} instances of {val} in {col}")
                    # Convert to NaN
                    df[col] = df[col].replace(val, np.nan)
        
        # Print summary
        print("\nSpecial value summary:")
        for val, count in special_value_counts.items():
            if count > 0:
                print(f"Total {val} values converted to NaN: {count}")
        
        return df

    def handle_missing_values(self, df, depression_df):
        """
        Handle missing values in the dataset:
        1. Convert special values to NaN
        2. Remove variables with high missingness
        3. Impute missing values:
           - Mean imputation for numerical variables
           - Mode imputation for categorical variables
        """
        print("\nHandling missing values...")
        
        # Step 1: Convert special values to NaN
        df = self.handle_special_values(df)
        
        # Step 2: Calculate missingness after special value conversion
        merged_df = pd.merge(df, depression_df[['src_subject_id']], 
                            on='src_subject_id', how='inner')
        missingness = merged_df.isnull().sum() / len(merged_df)
        
        # Print missing value statistics before removal
        print("\nMissing value statistics before removing high missingness variables:")
        high_missing_vars = missingness[missingness > self.missing_threshold]
        if not high_missing_vars.empty:
            print("\nVariables with high missingness:")
            for var, missing_pct in high_missing_vars.items():
                print(f"  - {var}: {missing_pct:.1%} missing")
        
        # Remove columns with high missingness
        keep_cols = missingness[missingness <= self.missing_threshold].index
        clean_df = df[keep_cols].copy()
        print(f"\nRemoved {len(df.columns) - len(keep_cols)} variables due to missingness")
        
        # Identify numeric and categorical columns for imputation
        numeric_cols = clean_df.select_dtypes(include=['int64', 'float64']).columns
        numeric_cols = [col for col in numeric_cols if col not in ['src_subject_id', 'depression_score']]
        
        categorical_cols = clean_df.select_dtypes(exclude=['int64', 'float64']).columns
        categorical_cols = [col for col in categorical_cols if col not in ['src_subject_id', 'depression_score']]
        
        # Calculate and print missing value statistics before imputation
        print("\nMissing values before imputation:")
        
        # Numeric variables
        numeric_missing = clean_df[numeric_cols].isnull().sum()
        total_numeric_missing = numeric_missing.sum()
        if total_numeric_missing > 0:
            print("\nNumeric variables:")
            print(f"Total missing values: {total_numeric_missing}")
            for col in numeric_cols:
                n_missing = numeric_missing[col]
                if n_missing > 0:
                    print(f"  - {col}: {n_missing} missing values ({n_missing/len(clean_df):.1%})")
        
        # Categorical variables
        categorical_missing = clean_df[categorical_cols].isnull().sum()
        total_categorical_missing = categorical_missing.sum()
        if total_categorical_missing > 0:
            print("\nCategorical variables:")
            print(f"Total missing values: {total_categorical_missing}")
            for col in categorical_cols:
                n_missing = categorical_missing[col]
                if n_missing > 0:
                    print(f"  - {col}: {n_missing} missing values ({n_missing/len(clean_df):.1%})")
        
        # Mean imputation for continuuos variables
        print("\nPerforming mean imputation for numeric variables...")
        for col in numeric_cols:
            if clean_df[col].isnull().any():
                mean_value = clean_df[col].mean()
                n_missing = clean_df[col].isnull().sum()
                clean_df[col] = clean_df[col].fillna(mean_value)
                print(f"Imputed {n_missing} missing values in {col} with mean: {mean_value:.2f}")
        
        #  Mode imputation for categorical variables
        print("\nPerforming mode imputation for categorical variables...")
        for col in categorical_cols:
            if clean_df[col].isnull().any():
                mode_value = clean_df[col].mode()[0]  # Get the most common value
                n_missing = clean_df[col].isnull().sum()
                clean_df[col] = clean_df[col].fillna(mode_value)
                print(f"Imputed {n_missing} missing values in {col} with mode: {mode_value}")
        
        # Verify no missing values 
        remaining_numeric_missing = clean_df[numeric_cols].isnull().sum().sum()
        remaining_categorical_missing = clean_df[categorical_cols].isnull().sum().sum()
        
        if remaining_numeric_missing > 0 or remaining_categorical_missing > 0:
            print(f"\nWarning: {remaining_numeric_missing + remaining_categorical_missing} missing values remain after imputation")
            if remaining_numeric_missing > 0:
                print(f"  - {remaining_numeric_missing} in numeric columns")
            if remaining_categorical_missing > 0:
                print(f"  - {remaining_categorical_missing} in categorical columns")
        else:
            print("\nAll missing values have been imputed")
        
        return clean_df
    
    def remove_low_variance_variables(self, df, continuous_vars):
        """
        Remove variables with low variance
        """
        print("\nRemoving low variance variables...")
        n_cols_before = len(df.columns)
        removed_vars = []
        
        for var in continuous_vars:
            if var in ['src_subject_id', 'depression_score']:
                continue
            if var not in df.columns:
                continue
            std = df[var].std()
            mean = df[var].mean()
            if mean != 0:
                cv = std / abs(mean)
                if cv <= self.cont_var_threshold:
                    df = df.drop(columns=[var])
                    removed_vars.append(f"{var} (continuous, cv={cv:.3f})")
        
        print(f"Removed {n_cols_before - len(df.columns)} low variance variables")
        if removed_vars:
            print("\nRemoved variables:")
            for var in removed_vars:
                print(f"  - {var}")
        
        return df
        
    def track_variables(self, df, stage_name):
        """
        Track which variables are present at each stage
        """
        print(f"\nVariables present after {stage_name}:")
        print(f"Total columns: {len(df.columns)}")
        numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
        print(f"Numeric columns: {len(numeric_cols)}")
        categorical_cols = df.select_dtypes(exclude=['int64', 'float64']).columns
        print(f"Categorical columns: {len(categorical_cols)}")
        return df

    def encode_categorical_variables(self, df, categorical_vars, binary_vars):
        """
        Encode categorical and binary variables
        """
        print("\nEncoding categorical variables...")
        
        # binary variables 
        for var in binary_vars:
            if var in df.columns:
                print(f"Using binary encoding for {var}")
                df[var] = (df[var] == df[var].value_counts().index[0]).astype(int)
                self.encoders[var] = 'binary'
        
        #  categorical variables
        for var in categorical_vars:
            if var in df.columns:
                if df[var].dtype in ['int64', 'float64'] or len(df[var].unique()) > 10:
                    print(f"Using ordinal encoding for {var}")
                    self.encoders[var] = OrdinalEncoder(handle_unknown='use_encoded_value', 
                                                      unknown_value=-1)
                    values = df[var].values.reshape(-1, 1)
                    df[var] = self.encoders[var].fit_transform(values)
                else:
                    print(f"Using one-hot encoding for {var}")
                    self.encoders[var] = OneHotEncoder(handle_unknown='ignore', sparse=False)
                    values = df[var].values.reshape(-1, 1)
                    encoded_values = self.encoders[var].fit_transform(values)
                    feature_names = [f"{var}_{i}" for i in range(encoded_values.shape[1])]
                    encoded_df = pd.DataFrame(encoded_values, columns=feature_names, index=df.index)
                    df = pd.concat([df.drop(columns=[var]), encoded_df], axis=1)
        
        return df
    
    def scale_numerical_variables(self, df, continuous_vars):
            """
            Scale numerical variables including depression score
            """
            print("\nScaling numerical variables...")
            self.scaler = StandardScaler()
            
            # Filter continuous_vars to only include those that still exist in the dataframe
            existing_continuous_vars = [var for var in continuous_vars if var in df.columns]
            
            # Add depression score to variables to scale
            vars_to_scale = existing_continuous_vars + ['depression_score']
            
            if not vars_to_scale:
                print("No continuous variables found to scale")
                return df
                
            print(f"Scaling {len(vars_to_scale)} continuous variables (including depression score)")
            df[vars_to_scale] = self.scaler.fit_transform(df[vars_to_scale])
            
            # Print statistics before and after scaling
            print("\nDepression score statistics before scaling:")
            print(df['depression_score'].describe())
            
            return df

    def transform_skewed_features(self, df, continuous_vars):
        """
        Apply log transformation to skewed features
        """
        print("\nApplying log transformation to skewed features...")
        
        # Filter continuous_vars to only include those that still exist in the dataframe
        existing_continuous_vars = [var for var in continuous_vars if var in df.columns]
        
        if not existing_continuous_vars:
            print("No continuous variables found to transform")
            return df
            
        for col in existing_continuous_vars:
            skewness = stats.skew(df[col].dropna())
            if abs(skewness) > 1:
                print(f"Log transforming {col} (skewness: {skewness:.2f})")
                df[col] = np.log1p(df[col] - df[col].min() + 1)
        
        return df

    def remove_correlated_features(self, df):
        """
        Remove highly correlated features
        """
        print("\nRemoving correlated features...")
        numeric_df = df.select_dtypes(include=['int64', 'float64'])
        corr_matrix = numeric_df.corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        to_drop = [column for column in upper.columns if any(upper[column] > self.correlation_threshold)]
        df = df.drop(columns=to_drop)
        print(f"Removed {len(to_drop)} highly correlated variables")
        return df

        
        return clean_df
    
    def remove_low_variance_variables(self, df, continuous_vars):
        """
        Remove variables with low variance
        """
        print("\nRemoving low variance variables...")
        n_cols_before = len(df.columns)
        removed_vars = []
        
        # Remove low variance continuous variables
        for var in continuous_vars:
            if var in ['src_subject_id', 'depression_score']:
                continue
            if var not in df.columns:
                continue
            std = df[var].std()
            mean = df[var].mean()
            if mean != 0:
                cv = std / abs(mean)
                if cv <= self.cont_var_threshold:
                    df = df.drop(columns=[var])
                    removed_vars.append(f"{var} (continuous, cv={cv:.3f})")
        
        print(f"Removed {n_cols_before - len(df.columns)} low variance variables")
        if removed_vars:
            print("\nRemoved variables:")
            for var in removed_vars:
                print(f"  - {var}")
        
        return df
        
    def track_variables(self, df, stage_name):
        """
        Track which variables are present at each stage
        """
        print(f"\nVariables present after {stage_name}:")
        print(f"Total columns: {len(df.columns)}")
        numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
        print(f"Numeric columns: {len(numeric_cols)}")
        categorical_cols = df.select_dtypes(exclude=['int64', 'float64']).columns
        print(f"Categorical columns: {len(categorical_cols)}")
        return df

    def encode_categorical_variables(self, df, categorical_vars, binary_vars):
        """
        Encode categorical and binary variables
        """
        print("\nEncoding categorical variables...")
        
        # binary variables 
        for var in binary_vars:
            if var in df.columns:
                print(f"Using binary encoding for {var}")
                df[var] = (df[var] == df[var].value_counts().index[0]).astype(int)
                self.encoders[var] = 'binary'
        
        #  categorical variables
        for var in categorical_vars:
            if var in df.columns:
                if df[var].dtype in ['int64', 'float64'] or len(df[var].unique()) > 10:
                    print(f"Using ordinal encoding for {var}")
                    self.encoders[var] = OrdinalEncoder(handle_unknown='use_encoded_value', 
                                                      unknown_value=-1)
                    values = df[var].values.reshape(-1, 1)
                    df[var] = self.encoders[var].fit_transform(values)
                else:
                    print(f"Using one-hot encoding for {var}")
                    self.encoders[var] = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
                    values = df[var].values.reshape(-1, 1)
                    encoded_values = self.encoders[var].fit_transform(values)
                    feature_names = [f"{var}_{i}" for i in range(encoded_values.shape[1])]
                    encoded_df = pd.DataFrame(encoded_values, columns=feature_names, index=df.index)
                    df = pd.concat([df.drop(columns=[var]), encoded_df], axis=1)
        
        return df
    
    def scale_numerical_variables(self, df, continuous_vars):
        """
        Scale numerical variables
        """
        print("\nScaling numerical variables...")
        self.scaler = StandardScaler()
        
        # Filter continuous_vars to only include those that still exist in the dataframe
        existing_continuous_vars = [var for var in continuous_vars if var in df.columns]
        
        if not existing_continuous_vars:
            print("No continuous variables found to scale")
            return df
            
        print(f"Scaling {len(existing_continuous_vars)} continuous variables")
        df[existing_continuous_vars] = self.scaler.fit_transform(df[existing_continuous_vars])
        
        return df

    def transform_skewed_features(self, df, continuous_vars):
        """
        Apply log transformation to skewed features
        """
        print("\nApplying log transformation to skewed features...")
        
        # Filter continuous_vars to only include those that still exist in the dataframe
        existing_continuous_vars = [var for var in continuous_vars if var in df.columns]
        
        if not existing_continuous_vars:
            print("No continuous variables found to transform")
            return df
            
        for col in existing_continuous_vars:
            skewness = stats.skew(df[col].dropna())
            if abs(skewness) > 1:
                print(f"Log transforming {col} (skewness: {skewness:.2f})")
                df[col] = np.log1p(df[col] - df[col].min() + 1)
        
        return df

    def remove_correlated_features(self, df):
        """
        Remove highly correlated features
        """
        print("\nRemoving correlated features...")
        numeric_df = df.select_dtypes(include=['int64', 'float64'])
        corr_matrix = numeric_df.corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        to_drop = [column for column in upper.columns if any(upper[column] > self.correlation_threshold)]
        df = df.drop(columns=to_drop)
        print(f"Removed {len(to_drop)} highly correlated variables")
        return df

    def remove_temporal_variables(self, df):
        """
        Remove variables containing temporal information (eventname, date, timestamp)
        """
        print("\nRemoving temporal variables...")
        temporal_keywords = ['eventname', 'date', 'timestamp']
        
        # Find columns containing temporal keywords
        temporal_cols = [col for col in df.columns 
                        if any(keyword in col.lower() for keyword in temporal_keywords)]
        
        # Remove temporal columns
        if temporal_cols:
            print(f"Removing {len(temporal_cols)} temporal variables:")
            for col in temporal_cols:
                print(f"  - {col}")
            df = df.drop(columns=temporal_cols)
        else:
            print("No temporal variables found")
            
        return df

    def preprocess(self, baseline_df, depression_df):
        """
        Run the complete preprocessing pipeline without train/test split
        """
        print("Starting data preprocessing pipeline...")
        
        # Step 1: Handle missing values
        df_before = baseline_df.copy()
        df = self.handle_missing_values(baseline_df, depression_df)
        
        # Step 1.5: Remove temporal variables
        df = self.remove_temporal_variables(df)
        
        # Identify columns for visualization
        numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
        numeric_cols = [col for col in numeric_cols if col not in ['src_subject_id', 'depression_score']]
        
        categorical_cols = df.select_dtypes(exclude=['int64', 'float64']).columns
        categorical_cols = [col for col in categorical_cols if col not in ['src_subject_id', 'depression_score']]
        
        # Step 2: Identify variable types
        continuous_vars, categorical_vars, binary_vars = self.identify_variable_types(df)
        print(f"\nIdentified {len(continuous_vars)} continuous, {len(categorical_vars)} categorical, "
              f"and {len(binary_vars)} binary variables")
        
        # Step 3: Remove low variance variables
        df = self.remove_low_variance_variables(df, continuous_vars)
        df = self.track_variables(df, "low variance removal")
            
        # Step 4: Encode categorical variables
        df = self.encode_categorical_variables(df, categorical_vars, binary_vars)
        
        # Step 5: Scale numerical variables
        df = self.scale_numerical_variables(df, continuous_vars)
        
        # Step 6: Transform skewed features
        df = self.transform_skewed_features(df, continuous_vars)
        
        # Step 7: Remove correlated features
        df = self.remove_correlated_features(df)
        
        # Step 8: Add depression scores
        df = pd.merge(df, depression_df[['src_subject_id', 'depression_score']], 
                     on='src_subject_id', how='inner')

        # Step 9: Scale depression score
        print("\nScaling depression score...")
        df['depression_score'] = self.scaler.fit_transform(df[['depression_score']])
        print("\nDepression score statistics after scaling:")
        print(df['depression_score'].describe())
        
        # Print final dataset information
        print("\nFinal preprocessed dataset:")
        print(f"Shape: {df.shape}")
        
        # Print final dataset information
        print("\nFinal preprocessed dataset:")
        print(f"Shape: {df.shape}")
        print("\nFeature types:")
        print(df.dtypes.value_counts())
        
        # Save the preprocessed dataset
        output_path = Path(self.output_dir) / 'preprocessed_data.csv'
        df.to_csv(output_path, index=False)
        print(f"\nPreprocessed data saved to: {output_path}")
        
        # Return the preprocessed data and preprocessing objects
        return {
            'data': df,
            'encoders': self.encoders,
            'scaler': self.scaler,
            'variable_types': {
                'continuous': continuous_vars,
                'categorical': categorical_vars,
                'binary': binary_vars
            }
        }

def main():
    # Load CBCL depression data
    print("Loading CBCL depression data...")
    depression_df = load_cbcl_depression()
    
    # Create depression categories
    print("\nCreating depression categories...")
    depression_df = create_depression_categories(depression_df)
    
    # Load baseline data for our subjects
    print("\nStarting baseline data loading...")
    baseline_df = load_baseline_data(depression_df['src_subject_id'])
    print(f"\nFinal baseline dataset shape: {baseline_df.shape}")
    
    # run preprocessing pipeline
    print("\nStarting preprocessing pipeline...")
    preprocessor = DataPreprocessor()
    preprocessed_data = preprocessor.preprocess(baseline_df, depression_df)
    
    print("\nDone!")

if __name__ == "__main__":
    main()

Loading CBCL depression data...
Original subjects: 10098
Subjects after cleaning: 10094

Depression score statistics:
count    10094.000000
mean         1.690113
std          2.520788
min          0.000000
25%          0.000000
50%          1.000000
75%          2.000000
max         21.000000
Name: depression_score, dtype: float64

Creating depression categories...

Depression Score Statistics:
Mean: 1.69
Std: 2.52
Q1 (25th percentile): 0.00
Q3 (75th percentile): 2.00
IQR: 2.00

Thresholds:
Z-score method (2 SD above mean): 6.73
IQR method (Q3 + 1.5*IQR): 5.00

Depression Categories Distribution:
depression_category
low       0.458094
medium    0.302952
high      0.238954
Name: proportion, dtype: float64

Starting baseline data loading...
Minimum required subjects per file: 8075

Processing linked-external-data...
  - led_l_seda_demo_c.csv: 8946 subjects (keeping)
  - led_l_lodes.csv: 9604 subjects (keeping)
  - led_l_denspop.csv: 9589 subjects (keeping)
  - led_l_urbsat.csv: 9604 subj